**Investigating Random Walks**

Gian Favero | ECSE 556 | December 1st, 2023

In [8]:
import pandas as pd
import numpy as np
from tqdm import tqdm
from helpers import *

First we load the .edge file associated with the “HumanNet Co-Expression of Human Genes (hn_HS_CX) network. The file will be cleaned up in such a way that only the relevant columns and values are kept.

In [9]:
edges_df = pd.read_csv('9606.hn_HS_CX.edge', sep='\t', header=None)
edges_df = edges_df.iloc[:, :3]
edges_df.columns = ['Node 1', 'Node 2', 'Weight']

Now we have to start forming the adjacency matrix that represents the network. We can get a set of every node in the network and then augment the dataset to ensure the network is undirected.

In [10]:
# Get set of all nodes
nodes = set(edges_df['Node 1'])
nodes = nodes.union(set(edges_df['Node 2']))

# Convert nodes to indices in edges_df
nodes_dict = dict(zip(nodes, range(len(nodes))))
edges_df['Node 1'] = edges_df['Node 1'].map(nodes_dict)
edges_df['Node 2'] = edges_df['Node 2'].map(nodes_dict)

# Initialize adjacency matrix
adj_mat = np.zeros((len(nodes), len(nodes)))

# Fill adjacency matrix
for i in tqdm(range(len(edges_df))):
    row = edges_df.iloc[i]
    adj_mat[int(row['Node 1']), int(row['Node 2'])] = row['Weight']

# If there are any self-loops, remove them
np.fill_diagonal(adj_mat, 0)

100%|██████████| 154387/154387 [00:08<00:00, 18763.26it/s]


In [11]:
# Adjust adjacency matrix to be symmetric, undirected
adj_mat = process_symmetric_entries(adj_mat)

# Find all subgraphs and get list of nodes that belong to subgraphs with less than 5 nodes
subgraphs = find_subgraphs(adj_mat)
nodes_to_remove = nodes_to_remove(subgraphs, 5)

# Remove nodes from adjacency matrix
adj_mat = np.delete(adj_mat, nodes_to_remove, axis=0)
adj_mat = np.delete(adj_mat, nodes_to_remove, axis=1)

# Normalize adjacency matrix by row
adj_mat = adj_mat / adj_mat.sum(axis=1, keepdims=True)

**No Restart**

Now, we initialize three random initial distribution vectors with only one non-zero entry. We run RW until convergence and output the final distributions to see if RW without restart preserves local information about the nodes.

In [12]:
# First random walk vector has only one entry of 1, rest are 0
rw_vec = np.zeros((adj_mat.shape[0], 1))
rw_vec1 = rw_vec.copy()
rw_vec1[0] = 1
rw_vec2 = rw_vec.copy()
rw_vec2[1500] = 1
rw_vec3 = rw_vec.copy()
rw_vec3[5000] = 1

# Perform random walk until there is small change in vector
eps = 1e-5
while True:
    rw_vec_new = adj_mat @ rw_vec1
    if np.linalg.norm(rw_vec_new - rw_vec1) < eps:
        break
    rw_vec1 = rw_vec_new
while True:
    rw_vec_new = adj_mat @ rw_vec2
    if np.linalg.norm(rw_vec_new - rw_vec2) < eps:
        break
    rw_vec2 = rw_vec_new
while True:
    rw_vec_new = adj_mat @ rw_vec3
    if np.linalg.norm(rw_vec_new - rw_vec3) < eps:
        break
    rw_vec3 = rw_vec_new

# See if the vectors are equal
print('Starting from random node 1 and random node 2:', np.allclose(rw_vec1, rw_vec2, atol=1e-2, rtol=1e-2))
print('Starting from random node 1 and random node 3:', np.allclose(rw_vec1, rw_vec3, atol=1e-2, rtol=1e-2))
print('Starting from random node 2 and random node 3:', np.allclose(rw_vec2, rw_vec3, atol=1e-2, rtol=1e-2))


Starting from random node 1 and random node 2: True
Starting from random node 1 and random node 3: True
Starting from random node 2 and random node 3: True


**With Restart**

In [13]:
''' Test code for adj matrix processing '''

# Define nodes and weights
node1 = ['A', 'B', 'C', 'D', 'B']
node2 = ['B', 'C', 'D', 'A', 'A']
weights = [1.0, 2.0, 3.0, 4.0, 5.0]

# Create dictionary
data = {'Node 1': node1, 'Node 2': node2, 'Weight': weights}

# Convert to DataFrame
test_df = pd.DataFrame(data)

print(test_df.head())

# Get set of all nodes
nodes = set(test_df['Node 1'])
nodes = nodes.union(set(test_df['Node 2']))

# Convert nodes to indices in edges_df
nodes_dict = dict(zip(nodes, range(len(nodes))))
test_df['Node 1'] = test_df['Node 1'].map(nodes_dict)
test_df['Node 2'] = test_df['Node 2'].map(nodes_dict)

print(test_df.head())

# Initialize adjacency matrix
test_adj = np.zeros((len(nodes), len(nodes)))

print(test_adj)

# Fill adjacency matrix
for i in range(len(test_df)):
    row = test_df.iloc[i]
    test_adj[int(row['Node 1']), int(row['Node 2'])] = row['Weight']

print(test_adj)

test_adj = process_symmetric_entries(test_adj)

print(test_adj)

# normalize adjacency matrix by row
test_adj = test_adj / test_adj.sum(axis=1, keepdims=True)

print(test_adj)

  Node 1 Node 2  Weight
0      A      B     1.0
1      B      C     2.0
2      C      D     3.0
3      D      A     4.0
4      B      A     5.0
   Node 1  Node 2  Weight
0       1       2     1.0
1       2       3     2.0
2       3       0     3.0
3       0       1     4.0
4       2       1     5.0
[[0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]]
[[0. 4. 0. 0.]
 [0. 0. 1. 0.]
 [0. 5. 0. 2.]
 [3. 0. 0. 0.]]
[[0. 4. 0. 3.]
 [4. 0. 3. 0.]
 [0. 3. 0. 2.]
 [3. 0. 2. 0.]]
[[0.         0.57142857 0.         0.42857143]
 [0.57142857 0.         0.42857143 0.        ]
 [0.         0.6        0.         0.4       ]
 [0.6        0.         0.4        0.        ]]
